In [11]:
# Request Body
# Section 0: imports
# import fast API, optional and uvicorn for using fastAPI
from fastapi import FastAPI
from typing import Optional
# import uvicorn, nest_asyncio to test and run API
import uvicorn 
import nest_asyncio
nest_asyncio.apply()

from pydantic import BaseModel

# what does this class do?
# by importing BaseModel idk what in the base model class
# idk why despite the vars name, price while type specified are not assigned
class Item(BaseModel):
    name: str
    description: Optional[str] = None
    price: float
    tax: Optional[float] = None

app = FastAPI()

#instance = Item()
#print(instance)


In [2]:
# Section 1: Simple Reuest Body Part

# Decorate FastAPI instance app with the method post, with str path param "/items/"
# define the Async function create_item(item: Item), taking param item of type class Item
# doesn't do much it returns Item?
@app.post("/items/")
async def create_item(item: Item):
    return item

In [3]:
# lets try to run a server
uvicorn.run(app, port=8001, host='localhost')

INFO:     Started server process [423565]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://localhost:8001 (Press CTRL+C to quit)
INFO:     127.0.0.1:60012 - "GET /docs HTTP/1.1" 200 OK
INFO:     127.0.0.1:60012 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     127.0.0.1:60024 - "POST /items/ HTTP/1.1" 422 Unprocessable Entity
INFO:     127.0.0.1:60026 - "POST /items/ HTTP/1.1" 422 Unprocessable Entity
INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [423565]


In [10]:
# this new post decoration can be tried
# where the async function create_item 
# takes a Item object and converts it to a dict
# adds the key 'price_with_tax' with value item.price + item.tax
@app.post("/items/")
async def create_item(item: Item):
    item_dict = item.dict()
    if item.tax:
        price_with_tax = item.price + item.tax
        item_dict.update({"price_with_tax": price_with_tax})
    return item_dict

# lets try to run a server
uvicorn.run(app, port=8001, host='localhost')

INFO:     Started server process [423565]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://localhost:8001 (Press CTRL+C to quit)
INFO:     127.0.0.1:60218 - "GET /docs HTTP/1.1" 200 OK
INFO:     127.0.0.1:60218 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     127.0.0.1:60222 - "POST /items/ HTTP/1.1" 200 OK
INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [423565]


In [12]:
# lets try put requests 
# understand that put is idempotent,
# so it makes sense that unique item_id path parameters
# would make sense. the query parameters are item_id int, and Item object
# ** unpackable ??????
@app.put("/items/{item_id}")
async def create_item(item_id: int, item: Item):
    return {"item_id": item_id, **item.dict()}

# lets try to run a server
uvicorn.run(app, port=8001, host='localhost')

INFO:     Started server process [423565]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://localhost:8001 (Press CTRL+C to quit)
INFO:     127.0.0.1:60364 - "GET /docs HTTP/1.1" 200 OK
INFO:     127.0.0.1:60364 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     127.0.0.1:60372 - "PUT /items/1 HTTP/1.1" 200 OK
INFO:     127.0.0.1:60376 - "PUT /items/12 HTTP/1.1" 200 OK
INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [423565]


In [ ]:
# integrating optional query parameters
# same unique put method with path param item_id
# and then using optional query parameter q,
# we can decide whether the json object "item_id" with item identification item_id:int has the parameter q added ?
@app.put("/items/{item_id}")
async def create_item(item_id: int, item: Item, q: Optional[str] = None):
    result = {"item_id": item_id, **item.dict()}
    if q:
        result.update({"q": q})
    return result